In [1]:
%pip uninstall -y opencv-python opencv-contrib-python
%pip install opencv-contrib-python-headless

Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /anaconda/envs/azureml_py310_sdkv2/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
from cv2 import dnn_superres
import os
import time

# --- CONFIGURATION ---
INPUT_DIR = "Combined_Study_Data"
OUTPUT_DIR = "Super_Resolution_Results"
MODEL_PATH = "SuperResolution/EDSR_x4.pb"

# LIMIT: Processing is slow (can take 5-10s per image on CPU). 
# Set to -1 to process ALL images, or keep at 5 for a quick test.
IMAGE_LIMIT = -1 

# Create output folder
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 1. Initialize the Super Resolution Object
print("--- Loading AI Model (this may take a moment)... ---")
sr = dnn_superres.DnnSuperResImpl_create()
sr.readModel(MODEL_PATH)
sr.setModel("edsr", 4) # Scale factor 4x

# 2. Get Images
files = [f for f in os.listdir(INPUT_DIR) if f.lower().endswith(('.jpg', '.png'))]
files_to_process = files[:IMAGE_LIMIT] if IMAGE_LIMIT > 0 else files

print(f"--- Starting Super Resolution on {len(files_to_process)} images ---")
print("Note: This is much slower than Bicubic Interpolation!")

for i, filename in enumerate(files_to_process):
    start_time = time.time()
    
    # Read Image
    path = os.path.join(INPUT_DIR, filename)
    img = cv2.imread(path) # Read in Color (BGR)
    
    if img is None: continue

    # Apply AI Upscaling
    # This single line runs the Neural Network
    result = sr.upsample(img)
    
    # Save Output
    save_name = f"SR_{filename}"
    cv2.imwrite(os.path.join(OUTPUT_DIR, save_name), result)
    
    elapsed = time.time() - start_time
    print(f"[{i+1}/{len(files_to_process)}] Processed {filename} ({elapsed:.2f}s)")

print(f"\nDone! Results saved to '{OUTPUT_DIR}'")

--- Loading AI Model (this may take a moment)... ---
--- Starting Super Resolution on 48 images ---
Note: This is much slower than Bicubic Interpolation!
[1/48] Processed headANT0_1100.jpg (27.95s)
[2/48] Processed headANT0_1217.jpg (27.05s)
[3/48] Processed headANT0_1349.jpg (27.02s)
[4/48] Processed headANT0_144.jpg (27.11s)
[5/48] Processed headANT0_1494.jpg (27.11s)
[6/48] Processed headANT0_154.jpg (26.98s)
[7/48] Processed headANT0_1569.jpg (27.36s)
[8/48] Processed headANT0_1576.jpg (26.88s)
[9/48] Processed headANT0_1604.jpg (26.97s)
[10/48] Processed headANT0_1632.jpg (26.88s)
[11/48] Processed headANT0_1666.jpg (26.79s)
[12/48] Processed headANT0_1714.jpg (26.96s)
[13/48] Processed headANT1_111.jpg (27.00s)
[14/48] Processed headANT1_1120.jpg (26.91s)
[15/48] Processed headANT1_1137.jpg (27.21s)
[16/48] Processed headANT1_1138.jpg (26.94s)
[17/48] Processed headANT1_1146.jpg (27.01s)
[18/48] Processed headANT1_1148.jpg (27.14s)
[19/48] Processed headANT1_1158.jpg (27.52s)
[20

In [3]:
import shutil
import os

# --- CONFIGURATION ---
# The folder created by the Super Resolution script
folder_to_zip = "Super_Resolution_Results"

# The name of the zip file you will download (without .zip)
output_filename = "Super_Resolution_Output"

print(f"--- Zipping folder: '{folder_to_zip}' ---")

if os.path.exists(folder_to_zip):
    # Create the zip file
    shutil.make_archive(output_filename, 'zip', folder_to_zip)
    
    print(f"✅ Success! Created '{output_filename}.zip'")
    print("👉 Go to the Files pane on the left, click Refresh (⟳), and download the zip file.")
else:
    print(f"❌ Error: Folder '{folder_to_zip}' not found. Did the previous script finish running?")

--- Zipping folder: 'Super_Resolution_Results' ---
✅ Success! Created 'Super_Resolution_Output.zip'
👉 Go to the Files pane on the left, click Refresh (⟳), and download the zip file.
